# Seq2seq - Mohamed Rostom GHARBI

In [1]:
import sys
import json
import operator
import numpy as np

from sklearn.decomposition import PCA

from gensim.models import KeyedVectors

from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras import backend as K
from keras.models import Model
from keras.backend.tensorflow_backend import _to_tensor
from keras.layers import Input, Embedding, Dropout, Bidirectional, GRU, TimeDistributed, Dense

Using TensorFlow backend.


In [2]:
path_root = ''
path_to_data = path_root + 'data/'

sys.path.insert(0, path_root)

from AttentionWithContext import AttentionWithContext

def bidir_gru(my_seq,n_units):
    '''
    just a convenient wrapper for bidirectional RNN with GRU units
    '''
    return Bidirectional(# adding a default GRU layer (https://keras.io/layers/recurrent/). 
                         GRU(units=n_units, activation='tanh', return_sequences=True),
                         merge_mode='concat', weights=None)(my_seq)
 
# = = = = = parameters = = = = =

n_units = 50
drop_rate = 0.5 
mfw_idx = 2 # index of the most frequent words in the dictionary. 
            # 0 is for the special padding token
            # 1 is for the special out-of-vocabulary token

padding_idx = 0
oov_idx = 1
batch_size = 32
nb_epochs = 6
my_optimizer = 'adam'
my_patience = 2 # for early stopping strategy


In [3]:
# = = = = = data loading = = = = =

my_docs_array_train = np.load(path_to_data + 'docs_train.npy')
my_docs_array_test = np.load(path_to_data + 'docs_test.npy')

my_labels_array_train = np.load(path_to_data + 'labels_train.npy')
my_labels_array_test = np.load(path_to_data + 'labels_test.npy')

# load dictionary of word indexes (sorted by decreasing frequency across the corpus)
with open(path_to_data + 'word_to_index.json', 'r') as my_file:
    word_to_index = json.load(my_file)

# invert mapping
index_to_word = dict((v,k) for k,v in word_to_index.items())

In [4]:
# = = = = = loading pretrained word vectors = = = = =

wvs = KeyedVectors.load(path_to_data + 'word_vectors.kv', mmap='r')
assert len(wvs.wv.vocab) == len(word_to_index) + 1 # vocab does not contain the OOV token

word_vecs = wvs.wv.syn0

pad_vec = np.random.normal(size=word_vecs.shape[1])

# add Gaussian vector on top of embedding matrix (padding vector)
word_vecs = np.insert(word_vecs,0,pad_vec,0)

print('embeddings created')

# reduce dimension with PCA (to reduce the number of parameters of the model)
my_pca = PCA(n_components=64)
embeddings_pca = my_pca.fit_transform(word_vecs)

print('embeddings compressed')

embeddings created
embeddings compressed


In [5]:
# = = = = = defining architecture = = = = =

# = = = sentence encoder

sent_ints = Input(shape=(my_docs_array_train.shape[2],)) # vec of ints of variable size

sent_wv = Embedding(input_dim=embeddings_pca.shape[0], # vocab size
                    output_dim=embeddings_pca.shape[1], # dimensionality of embedding space
                    weights=[embeddings_pca],
                    input_length=my_docs_array_train.shape[2],
                    trainable=True
                    )(sent_ints)

sent_wv_dr = Dropout(drop_rate)(sent_wv)

# using bidir_gru, AttentionWithContext with return_coefficients=True, and Dropout

sent_wa = bidir_gru(sent_wv_dr , n_units)

sent_att_vec , word_att_coeffs = AttentionWithContext(return_coefficients=True)(sent_wa)

sent_att_vec_dr = Dropout(drop_rate)(sent_att_vec)

sent_encoder = Model(sent_ints,sent_att_vec_dr)

# = = = document encoder

doc_ints = Input(shape=(my_docs_array_train.shape[1],my_docs_array_train.shape[2],))


# using TimeDistributed (https://keras.io/layers/wrappers/), bidir_gru, AttentionWithContext with return_coefficients=True, and Dropout
                  
sent_att_vec_dr = TimeDistributed(sent_encoder)(doc_ints)

doc_sa = bidir_gru(sent_att_vec_dr , n_units)

doc_att_vec , sent_att_coeffs = AttentionWithContext(return_coefficients=True)(doc_sa)

doc_att_vec_dr = Dropout(drop_rate)(doc_att_vec)

preds = Dense(units=1,
              activation='sigmoid')(doc_att_vec_dr)

model = Model(doc_ints,preds)

model.compile(loss='binary_crossentropy',
              optimizer = my_optimizer,
              metrics = ['accuracy'])

print('model compiled')

model compiled


In [6]:
# = = = = = training = = = = =

loading_pretrained = False

if not loading_pretrained:
    early_stopping = EarlyStopping(monitor='val_acc', # go through epochs as long as accuracy on validation set increases
                                   patience=my_patience,
                                   mode='max')
    
    # save model corresponding to best epoch
    checkpointer = ModelCheckpoint(filepath=path_to_data + 'model', 
                                   verbose=1, 
                                   save_best_only=True,
                                   save_weights_only=True)
    
    # 200s/epoch on CPU - reaches 84.38% accuracy in 2 epochs
    model.fit(my_docs_array_train, 
              my_labels_array_train,
              batch_size = batch_size,
              epochs = nb_epochs,
              validation_data = (np.array(my_docs_array_test), my_labels_array_test), #specifying validation data as tuple
              callbacks = [early_stopping,checkpointer])

else:
    model.load_weights(path_to_data + 'model')


Train on 25000 samples, validate on 25000 samples
Epoch 1/6
25000/25000 [==============================] - 240s 10ms/step - loss: 0.5480 - acc: 0.7056 - val_loss: 0.4296 - val_acc: 0.7997

Epoch 00001: val_loss improved from inf to 0.42964, saving model to data/model
Epoch 2/6
25000/25000 [==============================] - 227s 9ms/step - loss: 0.3631 - acc: 0.8428 - val_loss: 0.3741 - val_acc: 0.8350

Epoch 00002: val_loss improved from 0.42964 to 0.37406, saving model to data/model
Epoch 3/6
25000/25000 [==============================] - 232s 9ms/step - loss: 0.2809 - acc: 0.8844 - val_loss: 0.3685 - val_acc: 0.8403

Epoch 00003: val_loss improved from 0.37406 to 0.36848, saving model to data/model
Epoch 4/6
25000/25000 [==============================] - 235s 9ms/step - loss: 0.2207 - acc: 0.9109 - val_loss: 0.3811 - val_acc: 0.8421

Epoch 00004: val_loss did not improve from 0.36848
Epoch 5/6
25000/25000 [==============================] - 6181s 247ms/step - loss: 0.1833 - acc: 0.929

In [7]:
# = = = = = extraction of attention coefficients = = = = =

# define intermediate models (alternative to K.functions)


# defining a Model named 'get_word_att_coeffs' that extracts the attention coefficients over the words in a sentence
get_word_att_coeffs = Model(sent_ints, word_att_coeffs)

# define a Model named 'get_sent_attention_coeffs' that extracts the attention coefficients over the sentences in a document
# in each case, use the right inputs, and as outputs, the coefficients returned by the corresponding AttentionWithContext layer
get_sent_attention_coeffs = Model(doc_ints, sent_att_coeffs)
                            
                            
my_review = my_docs_array_test[-1:,:,:] # select last review
# convert integer review to text
index_to_word[1] = 'OOV'
my_review_text = [[index_to_word[idx] for idx in sent if idx in index_to_word] for sent in my_review.tolist()[0]]

In [8]:
# = = = attention over sentences in the document

sent_coeffs = get_sent_attention_coeffs.predict(my_review)
sent_coeffs = sent_coeffs[0,:,:]

for elt in zip(sent_coeffs[:,0].tolist(),[' '.join(elt) for elt in my_review_text]):
    print(round(elt[0]*100,2),elt[1])

# = = = attention over words in each sentence

my_review_tensor = _to_tensor(my_review,dtype='float32') # a layer, unlike a model, requires a TensorFlow tensor as input

# applying the 'get_word_att_coeffs' model over all the sentences in 'my_review_tensor', and store the results as 'word_coeffs'
word_coeffs = TimeDistributed(get_word_att_coeffs)(my_review_tensor)


word_coeffs = K.eval(word_coeffs) # shape = (1, 7, 30, 1): (batch size, nb of sents in doc, nb of words per sent, coeff)

word_coeffs = word_coeffs[0,:,:,0] # shape = (7, 30) (coeff for each word in each sentence)

word_coeffs = sent_coeffs * word_coeffs # re-weigh according to sentence importance

word_coeffs = np.round((word_coeffs*100).astype(np.float64),2)

word_coeffs_list = word_coeffs.tolist()

# match text and coefficients
text_word_coeffs = [list(zip(words,word_coeffs_list[idx][:len(words)])) for idx,words in enumerate(my_review_text)]

for sent in text_word_coeffs:
    [print(elt) for elt in sent]
    print('= = = =')

# sort words by importance within each sentence
text_word_coeffs_sorted = [sorted(elt,key=operator.itemgetter(1),reverse=True) for elt in text_word_coeffs]

for sent in text_word_coeffs_sorted:
    [print(elt) for elt in sent]
    print('= = = =')

4.29 There 's a sign on The Lost Highway that says : OOV SPOILERS OOV ( but you already knew that , did n't you ? )
5.6 Since there 's a great deal of people that apparently did not get the point of this movie , I 'd like to contribute my interpretation of why the plot
6.11 As others have pointed out , one single viewing of this movie is not sufficient .
19.94 If you have the DVD of MD , you can OOV ' by looking at David Lynch 's 'Top 10 OOV to OOV MD ' ( but only upon second
25.22 ; ) First of all , Mulholland Drive is downright brilliant .
22.99 A masterpiece .
15.85 This is the kind of movie that refuse to leave your head .
('There', 0.05)
("'s", 0.2)
('a', 0.24)
('sign', 0.13)
('on', 0.21)
('The', 0.1)
('Lost', 0.36)
('Highway', 0.83)
('that', 0.32)
('says', 0.22)
(':', 0.18)
('OOV', 0.22)
('SPOILERS', 0.26)
('OOV', 0.21)
('(', 0.16)
('but', 0.06)
('you', 0.05)
('already', 0.05)
('knew', 0.06)
('that', 0.07)
(',', 0.06)
('did', 0.02)
("n't", 0.03)
('you', 0.04)
('?', 0.05)
(')', 0.